In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


In [53]:
dfChurn = pd.read_csv("train.csv")
unseen = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")
data_dict = pd.read_csv("data_dictionary.csv")

print(dfChurn.shape)
print(unseen.shape)
print(sample.shape)
print(data_dict.shape)

(69999, 172)
(30000, 171)
(30000, 2)
(36, 2)


In [ ]:
dfChurn.head()

In [ ]:
dfChurn.describe(include='all')

In [ ]:
sns.distplot(churn['arpu_6'])

In [ ]:
# create column name list by types of columns
id_cols = ['mobile_number', 'circle_id']

date_cols = ['last_date_of_month_6',
             'last_date_of_month_7',
             'last_date_of_month_8',
             'date_of_last_rech_6',
             'date_of_last_rech_7',
             'date_of_last_rech_8',
             'date_of_last_rech_data_6',
             'date_of_last_rech_data_7',
             'date_of_last_rech_data_8'
            ]

cat_cols =  ['night_pck_user_6',
             'night_pck_user_7',
             'night_pck_user_8',
             'fb_user_6',
             'fb_user_7',
             'fb_user_8'
            ]

num_cols = [column for column in churn.columns if column not in id_cols + date_cols + cat_cols]

# print the number of columns in each list
print("#ID cols: %d\n#Date cols:%d\n#Numeric cols:%d\n#Category cols:%d" % (len(id_cols), len(date_cols), len(num_cols), len(cat_cols)))

# check if we have missed any column or not
print(len(id_cols) + len(date_cols) + len(num_cols) + len(cat_cols) == churn.shape[1])

### Drop columns

In [ ]:
print("Shape before dropping: ", churn.shape)
dfChurn = dfChurn.drop(id_cols + date_cols, axis=1)
print("Shape after dropping: ", churn.shape)

### Handling Missing Values

In [ ]:
missing = dfChurn.isnull().sum()*100/dfChurn.shape[0]

missing.sort_values(ascending=False)

#### Removing columns with more than threshold limit

In [ ]:
MISSING_THRESHOLD = 70

columns_greaterthan_threshold = missing[missing > MISSING_THRESHOLD].index

(columns_greaterthan_threshold)

In [ ]:
dfChurn = dfChurn.drop(columns_greaterthan_threshold, axis=1)

dfChurn.shape

#### Imputing values

In [57]:
zero_impute = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 
        'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8',
        'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8'
       ]

dfChurn[zero_impute] = dfChurn[zero_impute].fillna(0)

In [58]:
dfChurn[zero_impute] 

,total_rech_data_6,total_rech_data_7,total_rech_data_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,145.0,0.0,0.0,145.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7.0,7.0,6.0,175.0,191.0,142.0,25.0,41.0,25.0
...,...,...,...,...,...,...,...,...,...
69994,0.0,7.0,1.0,0.0,135.0,17.0,0.0,25.0,17.0
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,1.0,1.0,1.0,179.0,252.0,145.0,179.0,252.0,145.0
